In [1]:
!pip install git+https://github.com/mit-han-lab/smoothquant datasets

  Cloning https://github.com/mit-han-lab/smoothquant to /tmp/pip-req-build-rapbdaj8
  Running command git clone --filter=blob:none --quiet https://github.com/mit-han-lab/smoothquant /tmp/pip-req-build-rapbdaj8
  Resolved https://github.com/mit-han-lab/smoothquant to commit c61476d728e42ae0d8a35e7e78494edcac3237b5
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Created wheel for smoothquant: filename=smoothquant-0.0.0-py3-none-any.whl size=11948 sha256=b74e8ae4780b08d1a44942bf05aa34f2b2e88fe1191cb8e13a231ec18cc507b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-cds04zqa/wheels/12/3d/25/381605

In [2]:
import os

import torch
import torch.nn as nn
from transformers.models.llama.modeling_llama import (
    LlamaAttention,
    LlamaDecoderLayer,
    LlamaForCausalLM,
    LlamaMLP,
)
from transformers import LlamaTokenizer
from smoothquant.smooth import smooth_lm
from smoothquant.fake_quant import quantize_llama_like
import tqdm

In [3]:
class Evaluator:
    def __init__(self, dataset, tokenizer, device, n_samples=40):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        self.dataset = tokenizer(
            "\n\n".join(dataset["text"]), return_tensors="pt"
        ).input_ids.to(device)

        self.n_samples = n_samples

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        nlls = []
        print(self.dataset.shape)
        for i in tqdm.tqdm(range(self.n_samples), desc="Evaluating..."):
            batch = self.dataset[:, (i * 2048) : ((i + 1) * 2048)].to(model.device)
            with torch.no_grad():
                lm_logits = model(batch).logits
            shift_logits = lm_logits[:, :-1, :].contiguous().float()
            shift_labels = self.dataset[:, (i * 2048) : ((i + 1) * 2048)][:, 1:]
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
            )
            neg_log_likelihood = loss.float() * 2048
            nlls.append(neg_log_likelihood)

        return torch.exp(torch.stack(nlls).sum() / (self.n_samples * 2048))


In [4]:
from datasets import load_dataset

tokenizer = LlamaTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
evaluator = Evaluator(dataset, tokenizer, "cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/512k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


...

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (329290 > 2048). Running this sequence through the model will result in indexing errors


In [5]:
model_fp16 = LlamaForCausalLM.from_pretrained(
    "openlm-research/open_llama_3b_v2", torch_dtype=torch.float16, device_map="auto"
)

config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
ppl_fp16 = evaluator.evaluate(model_fp16)
print(f"Original model (fp16) perplexity: {ppl_fp16}")

torch.Size([1, 329290])


Evaluating...: 100%|██████████| 40/40 [00:17<00:00,  2.29it/s]


Original model (fp16) perplexity: 7.600046634674072


In [7]:
from smoothquant.calibration import get_act_scales
import functools

# https://github.com/mit-han-lab/smoothquant/blob/main/examples/generate_act_scales.py
def get_act_scales(model, tokenizer, dataset, num_samples=512, seq_len=512):
    model.eval()
    device = next(model.parameters()).device
    act_scales = {}

    def stat_tensor(name, tensor):
        hidden_dim = tensor.shape[-1]
        tensor = tensor.view(-1, hidden_dim).abs().detach()
        comming_max = torch.max(tensor, dim=0)[0].float().cpu()
        if name in act_scales:
            act_scales[name] = torch.max(act_scales[name], comming_max)
        else:
            act_scales[name] = comming_max

    def stat_input_hook(m, x, y, name):
        if isinstance(x, tuple):
            x = x[0]
        stat_tensor(name, x)

    hooks = []
    for name, m in model.named_modules():
        if isinstance(m, nn.Linear):
            hooks.append(
                m.register_forward_hook(functools.partial(stat_input_hook, name=name))
            )
    dataset = dataset.shuffle(seed=42)

    for i in tqdm.tqdm(range(num_samples)):
        input_ids = tokenizer(
            dataset[i]["text"], return_tensors="pt", max_length=seq_len, truncation=True
        ).input_ids.to(device)
        model(input_ids)

    for h in hooks:
        h.remove()

    return act_scales


act_scales = get_act_scales(
        model_fp16, tokenizer, load_dataset("wikitext", "wikitext-2-raw-v1", split="train"), num_samples=2048, seq_len=2048
)

100%|██████████| 2048/2048 [02:59<00:00, 11.40it/s]


In [8]:
smooth_lm(model_fp16, act_scales, 0.85)
model_smoothquant_w8a8 = quantize_llama_like(model_fp16)

In [9]:
ppl_smoothquant_w8a8 = evaluator.evaluate(model_smoothquant_w8a8)
print(f"SmoothQuant W8A8 quantized model perplexity: {ppl_smoothquant_w8a8}")

torch.Size([1, 329290])


Evaluating...: 100%|██████████| 40/40 [00:17<00:00,  2.26it/s]

SmoothQuant W8A8 quantized model perplexity: 7.640487194061279


In [10]:
# Naive quantization eval
model_fp16 = LlamaForCausalLM.from_pretrained(
    "openlm-research/open_llama_3b_v2", torch_dtype=torch.float16, device_map="auto"
)

In [11]:
model_smoothquant_w8a8 = quantize_llama_like(model_fp16)
ppl_smoothquant_w8a8 = evaluator.evaluate(model_smoothquant_w8a8)
print(f"Naive W8A8 quantized model perplexity: {ppl_smoothquant_w8a8}")

torch.Size([1, 329290])


Evaluating...: 100%|██████████| 40/40 [00:17<00:00,  2.34it/s]

Naive W8A8 quantized model perplexity: 7.709965229034424
